In [16]:
# pip install scapy-ssl_tls

# pip install scapy numpy matplotlib hilbertcurve

In [17]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scapy.all import rdpcap
from hilbertcurve.hilbertcurve import HilbertCurve
import ipaddress

import warnings
warnings.filterwarnings("ignore", category=UserWarning, message="WARNING: PcapNg: TLS Key Log available, but the TLS layer is not loaded! Scapy won't be able to decrypt the packets.")

# Define a function to convert IP address to integer
def ip_to_int(ip):
    return int(ipaddress.ip_address(ip))

# Define normalization function
def normalize(value, min_val, max_val):
    return int((value - min_val) / (max_val - min_val) * 255)

# Define a function to identify protocol, including SSL
def identify_protocol(packet):
    if packet.haslayer("TCP"):
        return 6  # TCP
    elif packet.haslayer("UDP"):
        return 17  # UDP
    elif packet.haslayer("ICMP"):
        return 1  # ICMP
    else:
        return 0  # OTHER

# Define a function to encode features into RGB channels
def combine_features_to_pixel(src_ip, des_ip, src_port, des_port, protocol):
    # Extracting the third byte from src_ip and des_ip
    r = (src_ip >> 16) & 0xFF  # Get the high byte of the src_ip
    g = (des_ip >> 16) & 0xFF  # Get the high byte of the des_ip
    # Combine src_port, des_port, and protocol into the blue channel
    b = ((src_port & 0xF0) >> 4) | (des_port & 0xF0) | (protocol & 0x0F)
    return (r, g, b)

# Function to process a single pcap file
def process_pcap_file(file_path, output_dir):
    packets = rdpcap(file_path)
    data = []

    for packet in packets:
        if packet.haslayer("IP"):
            src_ip = ip_to_int(packet["IP"].src)
            des_ip = ip_to_int(packet["IP"].dst)
            protocol = identify_protocol(packet)

            if packet.haslayer("TCP"):
                src_port = packet["TCP"].sport
                des_port = packet["TCP"].dport
            elif packet.haslayer("UDP"):
                src_port = packet["UDP"].sport
                des_port = packet["UDP"].dport
            else:
                src_port = 0
                des_port = 0
            data.append((src_ip, des_ip, src_port, des_port, protocol))

    if not data:
        return

    # Normalize feature values
    src_ips = [item[0] for item in data]
    des_ips = [item[1] for item in data]

    min_ip = min(src_ips + des_ips)
    max_ip = max(src_ips + des_ips)
    min_port = 0
    max_port = 65535
    min_protocol = 0
    max_protocol = 255

    normalized_data = []
    for src_ip, des_ip, src_port, des_port, protocol in data:
        norm_src_ip = normalize(src_ip, min_ip, max_ip)
        norm_des_ip = normalize(des_ip, min_ip, max_ip)
        norm_src_port = normalize(src_port, min_port, max_port)
        norm_des_port = normalize(des_port, min_port, max_port)
        norm_protocol = normalize(protocol, min_protocol, max_protocol)
        normalized_data.append((src_ip, des_ip, norm_src_port, norm_des_port, norm_protocol))

    # Map data to image using Hilbert curve
    order = 8  # Hilbert curve order, determines image size, 2^order * 2^order
    hilbert_curve = HilbertCurve(order, 2)
    image_size = 2**order
    image = np.zeros((image_size, image_size, 3), dtype=np.uint8)

    for i, (src_ip, des_ip, src_port, des_port, protocol) in enumerate(normalized_data):
        if i >= hilbert_curve.max_h:
            break
        x, y = hilbert_curve.point_from_distance(i)
        pixel = combine_features_to_pixel(src_ip, des_ip, src_port, des_port, protocol)
        image[x, y] = pixel

    # Save the image
    output_file_path = os.path.join(output_dir, os.path.basename(file_path) + '.png')
    plt.imsave(output_file_path, image)
    print(f"Saved image for {file_path} to {output_file_path}")

# Function to process all pcap files in a directory and its subdirectories
def process_all_pcaps(input_dir, output_dir):
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith('.pcap') or file.endswith('.pcapng'):
                file_path = os.path.join(root, file)
                # Check file size
                if os.path.getsize(file_path) < 2048:  # 2 KB = 2048 bytes
                    print(f"Skipping {file_path} as it is smaller than 2KB")
                    continue
                relative_path = os.path.relpath(root, input_dir)
                output_subdir = os.path.join(output_dir, relative_path)
                os.makedirs(output_subdir, exist_ok=True)
                process_pcap_file(file_path, output_subdir)

# Main function
def main():
    input_dir = 'data\\PCAPS'
    output_dir = 'Hilbert\\train'
    os.makedirs(output_dir, exist_ok=True)
    process_all_pcaps(input_dir, output_dir)

if __name__ == "__main__":
    main()


Saved image for data\PCAPS\AGENTTESLA\240610-m32w8agd2w.behavioral1.pcapng to Hilbert\train\AGENTTESLA\240610-m32w8agd2w.behavioral1.pcapng.png


Saved image for data\PCAPS\AGENTTESLA\240610-m32w8agd2w.behavioral2.pcapng to Hilbert\train\AGENTTESLA\240610-m32w8agd2w.behavioral2.pcapng.png


Saved image for data\PCAPS\AGENTTESLA\240610-m4d7jsgh74.behavioral1.pcap to Hilbert\train\AGENTTESLA\240610-m4d7jsgh74.behavioral1.pcap.png
Saved image for data\PCAPS\AGENTTESLA\240610-m4d7jsgh74.behavioral2.pcapng to Hilbert\train\AGENTTESLA\240610-m4d7jsgh74.behavioral2.pcapng.png
Saved image for data\PCAPS\AGENTTESLA\240610-m4e4vagh76.behavioral1.pcapng to Hilbert\train\AGENTTESLA\240610-m4e4vagh76.behavioral1.pcapng.png
Saved image for data\PCAPS\AGENTTESLA\240610-m4e4vagh76.behavioral2.pcapng to Hilbert\train\AGENTTESLA\240610-m4e4vagh76.behavioral2.pcapng.png
Saved image for data\PCAPS\AGENTTESLA\240610-m4ehbagd3z.behavioral1.pcapng to Hilbert\train\AGENTTESLA\240610-m4ehbagd3z.behavioral1.pcapng.png
Saved image for data\PCAPS\AGENTTESLA\240610-m4ehbagd3z.behavioral2.pcapng to Hilbert\train\AGENTTESLA\240610-m4ehbagd3z.behavioral2.pcapng.png
Saved image for data\PCAPS\AGENTTESLA\240610-m4fqdagh82.behavioral1.pcapng to Hilbert\train\AGENTTESLA\240610-m4fqdagh82.behavioral1.pcapng.

KeyboardInterrupt: 

In [ ]:
# !pip install bitstring


   ---------------------------------------- 0.0/71.7 kB ? eta -:--:--
   ---------------------------------------- 71.7/71.7 kB 3.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/126.0 kB ? eta -:--:--
   ---------------------------------------- 126.0/126.0 kB 3.6 MB/s eta 0:00:00


In [19]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scapy.all import rdpcap
from bitstring import BitArray
import ipaddress

import warnings
warnings.filterwarnings("ignore", category=UserWarning, message="WARNING: PcapNg: TLS Key Log available, but the TLS layer is not loaded! Scapy won't be able to decrypt the packets.")

# Define a function to convert IP address to integer
def ip_to_int(ip):
    return int(ipaddress.ip_address(ip))

# Define normalization function
def normalize(value, min_val, max_val):
    return int((value - min_val) / (max_val - min_val) * 255)

# Define a function to identify protocol, including SSL
def identify_protocol(packet):
    if packet.haslayer("TCP"):
        return 6  # TCP
    elif packet.haslayer("UDP"):
        return 17  # UDP
    elif packet.haslayer("ICMP"):
        return 1  # ICMP
    else:
        return 0  # OTHER

# Define a function to encode features into RGB channels
def combine_features_to_pixel(src_ip, des_ip, src_port, des_port, protocol):
    # Extracting the third byte from src_ip and des_ip
    r = (src_ip >> 16) & 0xFF  # Get the high byte of the src_ip
    g = (des_ip >> 16) & 0xFF  # Get the high byte of the des_ip
    # Combine src_port, des_port, and protocol into the blue channel
    b = ((src_port & 0xF0) >> 4) | (des_port & 0xF0) | (protocol & 0x0F)
    return (r, g, b)

# Function to interleave bits for Z-order curve
def interleave_bits(x, y, max_bits=16):
    z = BitArray()
    for i in range(max_bits):
        z.append(BitArray(uint=x >> i & 1, length=1))
        z.append(BitArray(uint=y >> i & 1, length=1))
    return z.uint

# Function to process a single pcap file
def process_pcap_file(file_path, output_dir):
    packets = rdpcap(file_path)
    data = []

    for packet in packets:
        if packet.haslayer("IP"):
            src_ip = ip_to_int(packet["IP"].src)
            des_ip = ip_to_int(packet["IP"].dst)
            protocol = identify_protocol(packet)

            if packet.haslayer("TCP"):
                src_port = packet["TCP"].sport
                des_port = packet["TCP"].dport
            elif packet.haslayer("UDP"):
                src_port = packet["UDP"].sport
                des_port = packet["UDP"].dport
            else:
                src_port = 0
                des_port = 0
            data.append((src_ip, des_ip, src_port, des_port, protocol))

    if not data:
        return

    # Normalize feature values
    src_ips = [item[0] for item in data]
    des_ips = [item[1] for item in data]

    min_ip = min(src_ips + des_ips)
    max_ip = max(src_ips + des_ips)
    min_port = 0
    max_port = 65535
    min_protocol = 0
    max_protocol = 255

    normalized_data = []
    for src_ip, des_ip, src_port, des_port, protocol in data:
        norm_src_ip = normalize(src_ip, min_ip, max_ip)
        norm_des_ip = normalize(des_ip, min_ip, max_ip)
        norm_src_port = normalize(src_port, min_port, max_port)
        norm_des_port = normalize(des_port, min_port, max_port)
        norm_protocol = normalize(protocol, min_protocol, max_protocol)
        normalized_data.append((src_ip, des_ip, norm_src_port, norm_des_port, norm_protocol))

    # Map data to image using Z-order curve
    image_size = 256
    image = np.zeros((image_size, image_size, 3), dtype=np.uint8)
    max_bits = int(np.log2(image_size))

    for i, (src_ip, des_ip, src_port, des_port, protocol) in enumerate(normalized_data):
        if i >= image_size**2:
            break
        z_value = interleave_bits(i % image_size, i // image_size, max_bits)
        x = z_value % image_size
        y = z_value // image_size
        if x < image_size and y < image_size:
            pixel = combine_features_to_pixel(src_ip, des_ip, src_port, des_port, protocol)
            image[x, y] = pixel

    # Save the image
    output_file_path = os.path.join(output_dir, os.path.basename(file_path) + '.png')
    plt.imsave(output_file_path, image)
    print(f"Saved image for {file_path} to {output_file_path}")

# Function to process all pcap files in a directory and its subdirectories
def process_all_pcaps(input_dir, output_dir):
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith('.pcap') or file.endswith('.pcapng'):
                file_path = os.path.join(root, file)
                # Check file size
                if os.path.getsize(file_path) < 2048:  # 2 KB = 2048 bytes
                    print(f"Skipping {file_path} as it is smaller than 2KB")
                    continue
                relative_path = os.path.relpath(root, input_dir)
                output_subdir = os.path.join(output_dir, relative_path)
                os.makedirs(output_subdir, exist_ok=True)
                process_pcap_file(file_path, output_subdir)

# Main function
def main():
    input_dir = 'data\\PCAPS'
    output_dir = 'z-order\\train'
    os.makedirs(output_dir, exist_ok=True)
    process_all_pcaps(input_dir, output_dir)

if __name__ == "__main__":
    main()


Saved image for data\PCAPS\AGENTTESLA\240610-m32w8agd2w.behavioral1.pcapng to z-order\train\AGENTTESLA\240610-m32w8agd2w.behavioral1.pcapng.png


Saved image for data\PCAPS\AGENTTESLA\240610-m32w8agd2w.behavioral2.pcapng to z-order\train\AGENTTESLA\240610-m32w8agd2w.behavioral2.pcapng.png


Saved image for data\PCAPS\AGENTTESLA\240610-m4d7jsgh74.behavioral1.pcap to z-order\train\AGENTTESLA\240610-m4d7jsgh74.behavioral1.pcap.png
Saved image for data\PCAPS\AGENTTESLA\240610-m4d7jsgh74.behavioral2.pcapng to z-order\train\AGENTTESLA\240610-m4d7jsgh74.behavioral2.pcapng.png
Saved image for data\PCAPS\AGENTTESLA\240610-m4e4vagh76.behavioral1.pcapng to z-order\train\AGENTTESLA\240610-m4e4vagh76.behavioral1.pcapng.png
Saved image for data\PCAPS\AGENTTESLA\240610-m4e4vagh76.behavioral2.pcapng to z-order\train\AGENTTESLA\240610-m4e4vagh76.behavioral2.pcapng.png
Saved image for data\PCAPS\AGENTTESLA\240610-m4ehbagd3z.behavioral1.pcapng to z-order\train\AGENTTESLA\240610-m4ehbagd3z.behavioral1.pcapng.png
Saved image for data\PCAPS\AGENTTESLA\240610-m4ehbagd3z.behavioral2.pcapng to z-order\train\AGENTTESLA\240610-m4ehbagd3z.behavioral2.pcapng.png
Saved image for data\PCAPS\AGENTTESLA\240610-m4fqdagh82.behavioral1.pcapng to z-order\train\AGENTTESLA\240610-m4fqdagh82.behavioral1.pcapng.

Saved image for data\PCAPS\AGENTTESLA\240610-nk13msha51.behavioral1.pcapng to z-order\train\AGENTTESLA\240610-nk13msha51.behavioral1.pcapng.png


Saved image for data\PCAPS\AGENTTESLA\240610-nk13msha51.behavioral2.pcapng to z-order\train\AGENTTESLA\240610-nk13msha51.behavioral2.pcapng.png


...

Saved image for data\PCAPS\ZLOADER\240523-bs9crsge2w.behavioral2.pcapng to z-order\train\ZLOADER\240523-bs9crsge2w.behavioral2.pcapng.png


Saved image for data\PCAPS\ZLOADER\240525-11e7bscg27.behavioral1.pcapng to z-order\train\ZLOADER\240525-11e7bscg27.behavioral1.pcapng.png
Saved image for data\PCAPS\ZLOADER\240525-11e7bscg27.behavioral2.pcapng to z-order\train\ZLOADER\240525-11e7bscg27.behavioral2.pcapng.png
Saved image for data\PCAPS\ZLOADER\240525-adwbxsga76.behavioral1.pcapng to z-order\train\ZLOADER\240525-adwbxsga76.behavioral1.pcapng.png


Saved image for data\PCAPS\ZLOADER\240525-b9mv4aad8t.behavioral1.pcapng to z-order\train\ZLOADER\240525-b9mv4aad8t.behavioral1.pcapng.png


Saved image for data\PCAPS\ZLOADER\240526-eq3c8afb64.behavioral1.pcapng to z-order\train\ZLOADER\240526-eq3c8afb64.behavioral1.pcapng.png


Saved image for data\PCAPS\ZLOADER\240526-eq3c8afb64.behavioral2.pcapng to z-order\train\ZLOADER\240526-eq3c8afb64.behavioral2.pcapng.png
Skipping data\PCAPS\ZLOADER\240530-bnfjwshb7y.behavioral1.pcapng as it is smaller than 2KB
Saved image for data\PCAPS\ZLOADER\240530-bnfjwshb7y.behavioral2.pcapng to z-order\train\ZLOADER\240530-bnfjwshb7y.behavioral2.pcapng.png
Skipping data\PCAPS\ZLOADER\240601-3gkc2sbc74.behavioral1.pcapng as it is smaller than 2KB
Saved image for data\PCAPS\ZLOADER\240601-3gkc2sbc74.behavioral2.pcapng to z-order\train\ZLOADER\240601-3gkc2sbc74.behavioral2.pcapng.png
Skipping data\PCAPS\ZLOADER\240601-m7bktsbe95.behavioral1.pcapng as it is smaller than 2KB
Saved image for data\PCAPS\ZLOADER\240601-m7bktsbe95.behavioral2.pcapng to z-order\train\ZLOADER\240601-m7bktsbe95.behavioral2.pcapng.png
Saved image for data\PCAPS\ZLOADER\240606-rnfsnafc5t.behavioral1.pcapng to z-order\train\ZLOADER\240606-rnfsnafc5t.behavioral1.pcapng.png
Saved image for data\PCAPS\ZLOADER\24